In [11]:
import re
import json
import pandas as pd

In [12]:
EXTRACT_DATE = '21-07-2020'

In [13]:
with open("Lærkeskellet.txt","r",encoding='utf8') as f:
    body = f.read()
    print(body)


Salgspriser - alle boliger - Lærkeskellet, 2000

Adresse                                         Type           Aktuel
                                                               m²
Lærkeskellet 18, 2000 Frederiksberg             Ejerlejlighed
                                                4.575.000      140
Fri handel     15-05-2018                       1.026.500      138

Fri handel     27-07-1994                       Ejerlejlighed  140
                                                4.800.000      140
Lærkeskellet 6, 2000 Frederiksberg              3.070.000
                                                907.133        140
Fri handel     16-11-2017                                      140
                                                Ejerlejlighed  140
Fri handel     31-01-2012                       3.495.000      138
                                                               138
Fri handel     18-11-1994                       Ejerlejlighed
                             

In [14]:
addresses = re.findall('Lærkeskellet \d{1,2},', body)
square_meters = re.findall(' \d{2,3}\n', body)
dates = re.findall('\d\d-\d\d-\d\d\d\d', body)
prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)

The pdf includes information on when the data was downlaoded from Boligsiden. This needs to be removed

I need to be able to find the addresses first and then find the dates for those addesses. Otherwise I can't line up the address, salesdate. 

The salesdate and price have the exact same number of rows, so those already lign up. But one or mole salesdates and prices can be part of the same address.

In [32]:
streetnames = [
    'Ahornskellet',
    'Askeskellet',
    'Birkeskellet',
    'Bøgeskellet',
    'Egeskellet',
    'Lindeskellet',
    'Lærkeskellet',
    'Pileskellet',
    'Plantanskellet',
    'Poppelskellet',
    'Tjørneskellet',
    "johannesvjensensalle"
]
rows = []
for street in streetnames:
    with open(f"{street}.txt","r", encoding='utf8') as f:
        body = f.read()
    addresses = re.findall(f'{street} ' + '\d{1,3}, [0-9a-z., ]*2000', body)
    address_chuncks = re.split(f'{street} '+ '\d{1,2},', body)

    prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)
    square_meters = re.findall(' \d{2,3}\n', body)
    
    price_cnt = 0
    for address, m2, text in zip(addresses, square_meters, address_chuncks[1:]):
        m2 = int(m2.replace('\n',''))
        dates = re.findall('\d\d-\d\d-\d\d\d\d', text)
        sale_types = re.findall('Familiehandel|Fri handel|I øvrigt|Auktion', text)
        try:
            dates.remove(EXTRACT_DATE)
        except ValueError:
            pass 

        for (date, sale_type) in zip(dates, sale_types):
            try:
                price = prices[price_cnt]
                price = int(price.replace('.',''))
                
            except:
                break

            m2_price = round(price/m2)
            street_number = address.replace(', 2000','').replace(street, '').replace(' ','').replace(',',' ').replace('.','')
            
            rows.append(
                [
                    street,
                    street_number,
                    date,
                    price,
                    m2,
                    m2_price,
                    sale_type
                ]
            )
            price_cnt += 1

df = pd.DataFrame(columns=[
        'streetName',
        'streetNumber',
        'salesDate',
        'price',
        'm2',
        'm2Price',
        'saleType'
    ],
    data=rows)
len(df)

304

In [33]:
df

,streetName,streetNumber,salesDate,price,m2,m2Price,saleType
0,Ahornskellet,14 2,22-10-2019,2450000,62,39516,Fri handel
1,Ahornskellet,14 2,15-12-2011,1530000,62,24677,Fri handel
2,Ahornskellet,14 2,02-04-2008,1425000,62,22984,Fri handel
3,Ahornskellet,6 2,06-10-2019,3900000,111,35135,Fri handel
4,Ahornskellet,6 2,15-11-2005,3295000,111,29685,Fri handel
5,Ahornskellet,16 st,22-04-2018,4225000,123,34350,Fri handel
6,Ahornskellet,16 st,09-05-2014,3000000,123,24390,Fri handel
7,Ahornskellet,16 st,02-11-2012,2900000,123,23577,Fri handel
8,Ahornskellet,16 st,18-04-2007,3000000,123,24390,Fri handel
9,Ahornskellet,6 st,24-10-2017,4100000,123,33333,Fri handel


In [34]:
df[
    (df.streetName == "johannesvjensensalle") &
    (df.streetNumber == "14 2")
]

,streetName,streetNumber,salesDate,price,m2,m2Price,saleType


We will keep only "fri handel" for now. 

In [35]:
df[df.saleType=="Fri handel"].to_csv('../../data/real-estate/sales.csv')

In [24]:
addressesJson = {}
streetJsons = []
for street in df.streetName.unique():
    street_df = df[df.streetName==street]
    streetJson = {
        'street':street,
        'numbers': list(street_df.streetNumber.unique())
    }
    streetJsons.append(streetJson)
addressesJson['streets'] = streetJsons 

In [25]:
with open('../../data/real-estate/addresses.json', 'w', encoding='utf8') as json_file:
  json.dump(addressesJson, json_file)


In [26]:
pd.DataFrame(
    df.m2.unique(),
    columns=['m2']
).to_csv('../../data/real-estate/squaremeters.csv')